**데이터셋:** AI-hub 한식이미지 데이터셋

**데이터셋 구조:** 이미지파일과 properties파일(이미지에 대한 좌표값이 있는 파일 but 전체가 있지 않음)

properties파일에 존재하지 않는 좌표는 이미지 전체값으로 설정하여 사용

### 1. YOLOv8 설치

In [ ]:
!pip install ultralytics

### 2. Google Drive 마운트 및 경로 설정

In [ ]:
from google.colab import drive
import os
import zipfile

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 경로 설정
dataset_zip_path = '/content/drive/MyDrive/YOLO/dataset.zip'  # 압축된 dataset 폴더 경로
output_dir = '/content/YOLO/dataset'  # 압축 해제할 경로
yolo_dataset_path = '/content/YOLO/yolo_dataset'  # YOLO 형식으로 변환한 데이터를 저장할 경로
train_path = '/content/YOLO/yolo_dataset/train'  # 학습 데이터 경로
val_path = '/content/YOLO/yolo_dataset/val'  # 검증 데이터 경로

# 폴더 생성
os.makedirs(output_dir, exist_ok=True)
os.makedirs(yolo_dataset_path, exist_ok=True)
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

Mounted at /content/drive


### 3. 데이터셋 압축 해제

In [ ]:
# 압축 풀기
with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

### 4. YOLO형식으로 데이터셋 경로 설정(image와 txt가 매칭될때만 복사)

In [ ]:
import shutil

# 압축 풀린 데이터셋 경로 설정
dataset_path = os.path.join(output_dir, 'dataset/kfood')  # kfood 폴더 경로
output_txts_path = os.path.join(output_dir, 'dataset/empty_labeling')  # empty_labeling 폴더 경로

# 대분류 폴더 순회
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)

    if os.path.isdir(category_path):
        # 중분류 폴더 순회
        for subcategory in os.listdir(category_path):
            subcategory_path = os.path.join(category_path, subcategory)

            if os.path.isdir(subcategory_path):
                # 이미지와 properties 파일을 포함한 파일 탐색
                for img_file in os.listdir(subcategory_path):
                    if img_file.endswith(('.jpg', '.png', '.jpeg')):
                        img_path = os.path.join(subcategory_path, img_file)

                        # 좌표 파일을 empty_labeling에서 찾기
                        base_name = os.path.splitext(img_file)[0]
                        txt_file = f'{base_name}.txt'
                        txt_file_path = os.path.join(output_txts_path, txt_file)

                        # 텍스트 파일이 있는 경우에만 복사
                        if os.path.exists(txt_file_path):
                            print(f"Copying image: {img_path}")
                            shutil.copy(img_path, yolo_dataset_path)
                            print(f"Copying label: {txt_file_path}")
                            shutil.copy(txt_file_path, yolo_dataset_path)
                        else:
                            print(f"Label not found for: {img_file}, skipping image.")

### 5. 데이터셋 분할
8:2 비율로 학습/검증 데이터 나눔

In [ ]:
import random

# 이미지 파일 리스트 가져오기
image_files = [f for f in os.listdir(yolo_dataset_path) if f.endswith('.jpg')]

# 데이터를 80:20 비율로 나누기
random.shuffle(image_files)
split_idx = int(0.8 * len(image_files))
train_files = image_files[:split_idx]
val_files = image_files[split_idx:]

# 학습 데이터와 검증 데이터로 파일 복사
for file in train_files:
    shutil.copy(os.path.join(yolo_dataset_path, file), train_path)
    # 해당 이미지에 맞는 .txt 파일도 복사
    txt_file = file.replace('.jpg', '.txt')
    if os.path.exists(os.path.join(yolo_dataset_path, txt_file)):
        shutil.copy(os.path.join(yolo_dataset_path, txt_file), train_path)

for file in val_files:
    shutil.copy(os.path.join(yolo_dataset_path, file), val_path)
    # 해당 이미지에 맞는 .txt 파일도 복사
    txt_file = file.replace('.jpg', '.txt')
    if os.path.exists(os.path.join(yolo_dataset_path, txt_file)):
        shutil.copy(os.path.join(yolo_dataset_path, txt_file), val_path)

### 6. 데이터 구성 파일 생성
data.yaml 파일을 생성하여 YOLO 모델 학습에 필요한 데이터 정보를 설정

In [ ]:
data_yaml = f"""
train: {train_path}  # train 데이터 경로
val: {val_path}      # validation 데이터 경로

# 클래스 수
nc: 150  # 총 클래스 수

# 클래스 이름
names:
  - 갈비구이
  - 갈치구이
  - 고등어구이
  - 곱창구이
  - 닭갈비
  - 더덕구이
  - 떡갈비
  - 불고기
  - 삼겹살
  - 장어구이
  - 조개구이
  - 조기구이
  - 황태구이
  - 훈제오리
  - 계란국
  - 떡국_만두국
  - 무국
  - 미역국
  - 북엇국
  - 시래기국
  - 육개장
  - 콩나물국
  - 과메기
  - 양념치킨
  - 젓갈
  - 콩자반
  - 편육
  - 피자
  - 후라이드치킨
  - 갓김치
  - 깍두기
  - 나박김치
  - 무생채
  - 배추김치
  - 백김치
  - 부추김치
  - 열무김치
  - 오이소박이
  - 총각김치
  - 파김치
  - 가지볶음
  - 고사리나물
  - 미역줄기볶음
  - 숙주나물
  - 시금치나물
  - 애호박볶음
  - 경단
  - 꿀떡
  - 송편
  - 만두
  - 라면
  - 막국수
  - 물냉면
  - 비빔냉면
  - 수제비
  - 열무국수
  - 잔치국수
  - 짜장면
  - 짬뽕
  - 쫄면
  - 칼국수
  - 콩국수
  - 꽈리고추무침
  - 도라지무침
  - 도토리묵
  - 잡채
  - 콩나물무침
  - 홍어무침
  - 회무침
  - 김밥
  - 김치볶음밥
  - 누룽지
  - 비빔밥
  - 새우볶음밥
  - 알밥
  - 유부초밥
  - 잡곡밥
  - 주먹밥
  - 감자채볶음
  - 건새우볶음
  - 고추장진미채볶음
  - 두부김치
  - 떡볶이
  - 라볶이
  - 멸치볶음
  - 소세지볶음
  - 어묵볶음
  - 오징어채볶음
  - 제육볶음
  - 주꾸미볶음
  - 보쌈
  - 수정과
  - 식혜
  - 간장게장
  - 양념게장
  - 깻잎장아찌
  - 떡꼬치
  - 감자전
  - 계란말이
  - 계란후라이
  - 김치전
  - 동그랑땡
  - 생선전
  - 파전
  - 호박전
  - 곱창전골
  - 갈치조림
  - 감자조림
  - 고등어조림
  - 꽁치조림
  - 두부조림
  - 땅콩조림
  - 메추리알장조림
  - 연근조림
  - 우엉조림
  - 장조림
  - 코다리조림
  - 전복죽
  - 호박죽
  - 김치찌개
  - 닭계장
  - 동태찌개
  - 된장찌개
  - 순두부찌개
  - 갈비찜
  - 계란찜
  - 김치찜
  - 꼬막찜
  - 닭볶음탕
  - 수육
  - 순대
  - 족발
  - 찜닭
  - 해물찜
  - 갈비탕
  - 감자탕
  - 곰탕_설렁탕
  - 매운탕
  - 삼계탕
  - 추어탕
  - 고추튀김
  - 새우튀김
  - 오징어튀김
  - 약과
  - 약식
  - 한과
  - 멍게
  - 산낙지
  - 물회
  - 육회
"""

# YAML 파일 저장
yaml_file_path = '/content/YOLO/data.yaml'
with open(yaml_file_path, 'w') as file:
    file.write(data_yaml)

print(f"data.yaml 파일이 {yaml_file_path}에 저장되었습니다.")

data.yaml 파일이 /content/YOLO/data.yaml에 저장되었습니다.


### 7. YOLOv8n 모델 학습

In [ ]:
from ultralytics import YOLO

# YOLOv8 모델 불러오기
model = YOLO('yolov8n.pt')

# 모델 학습
model.train(
    data=yaml_file_path,  # 데이터 설정 경로
    epochs=50,            # 에포크 수
    imgsz=640,            # 이미지 사이즈
    batch=16,             # 배치 사이즈
    name='train_empty_labeling',  # 학습 결과가 저장될 이름
    save=True,            # 모델 저장 여부
    save_period=-1,       # 모델 저장 주기 (기본 값은 마지막만 저장)
    workers=8,            # 데이터 로드 워커 수
    project='/content/YOLO/runs/detect_v2',  # 결과 저장 경로
    exist_ok=True         # 기존 폴더에 덮어쓰기 허용
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 275MB/s]


Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/YOLO/runs/detect_v2, name=train_empty_labeling, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cr

100%|██████████| 22.2M/22.2M [00:00<00:00, 36.1MB/s]


Overriding model.yaml nc=80 with nc=150

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralyti

...

      50/50      2.75G     0.3396     0.5273      1.191          2        640: 100%|██████████| 7082/7082 [11:24<00:00, 10.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 887/887 [01:54<00:00,  7.76it/s]


                   all      28371      28371      0.832      0.777      0.853      0.756

50 epochs completed in 11.168 hours.
Optimizer stripped from /content/YOLO/runs/detect_v2/train_empty_labeling/weights/last.pt, 6.9MB
Optimizer stripped from /content/YOLO/runs/detect_v2/train_empty_labeling/weights/best.pt, 6.9MB

Validating /content/YOLO/runs/detect_v2/train_empty_labeling/weights/best.pt...
Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 3,355,874 parameters, 0 gradients, 9.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 887/887 [01:56<00:00,  7.60it/s]
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:470: UserWarning: Glyph 44040 (\N{HANGUL SYLLABLE GAL}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:470: UserWarning: Glyph 48708 (\N{HANGUL SYLLABLE BI}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:470: UserWarning: Glyph 44396 (\N{HANGUL SYLLABLE GU}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:470: UserWarning: Glyph 51060 (\N{HANGUL SYLLABLE I}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:495: UserWarning: Glyph 44040 (\N{HANGUL SYLLABLE GAL})

                   all      28371      28371      0.832      0.777      0.853      0.756
                  갈비구이      28371      28371      0.832      0.777      0.853      0.756
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /content/YOLO/runs/detect_v2/train_empty_labeling


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e9a66b34b80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### 8. 학습된 모델 로드 및 성능 평가

In [ ]:
from ultralytics import YOLO

# 학습된 모델 불러오기 (best 모델을 사용)
model = YOLO('/content/YOLO/runs/detect_v2/train_empty_labeling/weights/best.pt')  # 경로는 학습된 모델의 위치에 맞게 수정

# 검증 데이터로 성능 평가
results = model.val()

# 결과 출력
print(results)

Ultralytics YOLOv8.2.100 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 168 layers, 3,355,874 parameters, 0 gradients, 9.7 GFLOPs


val: Scanning /content/YOLO/yolo_dataset/val.cache... 29519 images, 0 backgrounds, 1196 corrupt: 100%|██████████| 29567/29567 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/YOLO/yolo_dataset/val/Img_000_0113.jpg: ignoring corrupt image/label: negative label values [ -0.0084034]
val: WARNING ⚠️ /content/YOLO/yolo_dataset/val/Img_000_0145.jpg: ignoring corrupt image/label: invalid image format GIF. Supported formats are:
images: {'jpg', 'jpeg', 'mpo', 'png', 'webp', 'tiff', 'pfm', 'dng', 'tif', 'bmp'}
videos: {'wmv', 'webm', 'ts', 'asf', 'avi', 'mpg', 'mkv', 'gif', 'mpeg', 'mp4', 'm4v', 'mov'}
val: WARNING ⚠️ /content/YOLO/yolo_dataset/val/Img_000_0197.jpg: ignoring corrupt image/label: invalid image format GIF. Supported formats are:
images: {'jpg', 'jpeg', 'mpo', 'png', 'webp', 'tiff', 'pfm', 'dng', 'tif', 'bmp'}
videos: {'wmv', 'webm', 'ts', 'asf', 'avi', 'mpg', 'mkv', 'gif', 'mpeg', 'mp4', 'm4v', 'mov'}
val: WARNING ⚠️ /content/YOLO/yolo_dataset/val/Img_000_0211.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/YOLO/yolo_dataset/val/Img_000_0230.jpg: ignoring corrupt image/label: negative label values [  -0.046656]
v


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1774/1774 [02:03<00:00, 14.39it/s]
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:470: UserWarning: Glyph 44040 (\N{HANGUL SYLLABLE GAL}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:470: UserWarning: Glyph 48708 (\N{HANGUL SYLLABLE BI}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:470: UserWarning: Glyph 44396 (\N{HANGUL SYLLABLE GU}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:470: UserWarning: Glyph 51060 (\N{HANGUL SYLLABLE I}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:495: UserWarning: Glyph 44040 (\N{HANGUL SYLLABLE GA

                   all      28371      28371      0.832      0.777      0.853      0.756
                  갈비구이      28371      28371      0.832      0.777      0.853      0.756
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e999e453550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026, 

### 9. YOLO 학습 결과 백업

In [ ]:
import shutil

# YOLO 학습 결과가 있는 runs 폴더 경로
source_folder = '/content/YOLO/runs'
# 구글 드라이브에 저장할 경로
destination_folder = '/content/drive/MyDrive/YOLO/runs_backup'

# 폴더를 복사
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/YOLO/runs_backup'

### 10. 모델 성능 결과 확인

In [ ]:
# 결과 객체 확인
print(results)

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e9a5bc461a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

- runs파일 검토 후 전체 id값이 0으로 되어있는 것을 확인 → id값 수정(empty_labeling.ipynb의 두번째 코드)